In [38]:
import pandas as pd
import numpy as np
import scipy.sparse
import csv
import sys
from scipy.sparse import csr_matrix

In [39]:
import pickle
import timeit
def save_obj(obj, name ):
    with open('dumps/'+name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('dumps/'+name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [40]:
csv.field_size_limit(sys.maxsize)
csv.field_size_limit(sys.maxsize)

9223372036854775807

In [41]:
f = open('user.csv', 'r')
reader = csv.reader(f)
reader = csv.DictReader(f)
user_id_list = []
for i,row in enumerate(reader):
        user_id_list.append(row['user_id'])

In [42]:
print (len(set(user_id_list)),len(user_id_list))

(1326101, 1326101)


In [43]:
f = open('business.csv', 'r')
reader = csv.reader(f)
reader = csv.DictReader(f)
business_id_list = []
for i,row in enumerate(reader):
        business_id_list.append(row['business_id'])

In [44]:
print (len(set(business_id_list)),len(business_id_list))

(174567, 174567)


In [8]:
# lat_items = open('files/lat_item.csv').readlines()
# item_dict = {}
# for i, item in enumerate(lat_items[1:]):
#     item_parts = item.strip().split(',')
#     item_dict[int(item_parts[0])] = np.asarray((item_parts[-1])[2:-2].split(),dtype = np.float32) 

In [9]:
# R.shape (1326101, 174567)

In [45]:
lat_items = open('files/lat_item.csv').readlines()
item_dict = {}
print (lat_items[0])
for i, item in enumerate(lat_items[1:]):
    item_parts = item.strip().split(',')
    item_dict[int(item_parts[0])] = np.asarray((item_parts[-1])[2:-2].split(),dtype = np.float32) 
    
rank = np.asarray((item_parts[-1])[2:-2].split(),dtype = np.float32).shape[0]


item_mat = np.zeros((174567,rank))
z = np.zeros((rank,))
zer = 0
nzer = 0
for i in range(item_mat.shape[0]):
    if i+1 in item_dict:
        item_mat[i,:] = item_dict[i+1]
        nzer +=1
    else:
        item_mat[i,:] = z
        zer+=1
print (zer,nzer)
        

business_id,linear_terms,factors

(4369, 170198)


In [46]:
lat_users = open('files/lat_user.csv').readlines()
user_dict = {}
print (lat_users[0])
for i, item in enumerate(lat_users[1:]):
    item_parts = item.strip().split(',')
    user_dict[int(item_parts[0])] = np.asarray((item_parts[-1])[2:-2].split(),dtype = np.float32) 
    
rank = np.asarray((item_parts[-1])[2:-2].split(),dtype = np.float32).shape[0]
zer = 0
nzer = 0

user_mat = np.zeros((1326101,rank))
z = np.zeros((rank,))
for i in range(user_mat.shape[0]):
    if i+1 in user_dict:
        user_mat[i,:] = user_dict[i+1]
        nzer +=1
    else:
        user_mat[i,:] = z
        zer+=1
print (zer,nzer)

user_id,linear_terms,factors

(77319, 1248782)


In [12]:
user_mat.dtype

dtype('float64')

In [ ]:
np.save('files/user_mat',user_mat)
np.save('files/item_mat',item_mat)

In [3]:
user_mat.reshape(user_mat.shape[0]*user_mat.shape[1]).tofile('files/U.csv.bin')
item_mat.reshape(item_mat.shape[0]*item_mat.shape[1]).tofile('files/V.csv.bin')

NameError: name 'user_mat' is not defined

In [11]:
A.shape

(1326101, 50)

In [4]:
np.fromfile('files/U.csv.bin').shape

(66305050,)

In [6]:
rank =50

In [7]:
A = np.fromfile('files/U.csv.bin').reshape(1326101,rank)

In [10]:
np.linalg.norm(A-user_mat)

0.0

In [8]:
save_obj(business_id_list, 'business_id_list' )
save_obj(user_id_list, 'user_id_list' )

In [9]:
def itemize(data,s):
#     _,indx = np.unique(data,return_index=True)
#     u = data[np.sort(indx)]
    u = load_obj(s)
    n_data = len(u)
    new_indx = np.arange(n_data)
    d = dict(zip(u,new_indx))
    data_indx = np.zeros(data.shape, dtype=np.int32)
    for i in range(data_indx.shape[0]):
        data_indx[i] = d[data[i]]
    return data_indx, n_data

def load_data():
    data = pd.read_csv('review.csv')
    data = data.drop(['funny', 'review_id', 'text', 'date', 'useful', 'cool'], axis=1)
    data.drop_duplicates(inplace=True)
    rows, cols, stars = np.array(data['user_id']), np.array(data['business_id']), np.array(data['stars'],dtype=np.uint8)
    # itemize users and items
    row_indx, n_users = itemize(rows,'user_id_list')
    col_indx, n_items = itemize(cols,'business_id_list' )
    return scipy.sparse.csr_matrix((stars,(row_indx, col_indx)), dtype=np.uint8, shape=(n_users,n_items))

# R = scipy.sparse.csr_matrix((stars,(row_indx, col_indx)), dtype=np.uint8, shape=(n_users,n_items))
R = load_data()

In [10]:
tr_list = R.nonzero()
No_entries = len(tr_list[0])
print (No_entries)

5261667


In [11]:
R.shape  # (users, businesses) 

(1326101, 174567)

In [12]:
import random
import copy

In [13]:
cold_start_users = random.sample(range(R.shape[0]),int(R.shape[0]*0.025))
cold_start_items = random.sample(range(R.shape[1]),int(R.shape[1]*0.025))

In [14]:
cold_start_users = set(cold_start_users)
cold_start_items = set(cold_start_items)

In [15]:
R_copy = copy.deepcopy(R)

In [16]:
cold_start_user_triplet = []
cold_start_item_triplet = []

In [17]:
print (len(tr_list[0]))
total = len(tr_list[0])

5261667


In [18]:
print (len(cold_start_users),len(cold_start_items))

(33152, 4364)


In [19]:
for i in zip(tr_list[0],tr_list[1]):
    if i[0] in cold_start_users:
        cold_start_user_triplet.append(i)
        R_copy[i[0],i[1]] = 0
    if i[1] in cold_start_items:
        cold_start_item_triplet.append(i)
        R_copy[i[0],i[1]] = 0

In [20]:
train_warmTest_TMP = R_copy.nonzero()
print (len(train_warmTest_TMP[0]))
train_warmTest = [(i[0],i[1]) for i in zip(train_warmTest_TMP[0],train_warmTest_TMP[1])]
warm_test = random.sample(train_warmTest,200000)
for i in warm_test:
    R_copy[i[0],i[1]] = 0

train = R_copy.nonzero()

train = [(i[0],i[1]) for i in zip(train[0],train[1])]

print (len(train))

5001323
4801323


In [21]:
print ('train',len(train))
print ('warm_test',len(warm_test))
print ('cold_item',len(cold_start_item_triplet))
print ('cold_user',len(cold_start_user_triplet))

('train', 4801323)
('warm_test', 200000)
('cold_item', 129248)
('cold_user', 134378)


In [28]:
with open('files/test_cold_item.csv','w') as f:
    for i in cold_start_item_triplet:
        f.write(   str(i[0]+1)+','+str(i[1]+1)+','+str(R[i[0],i[1]])+ ','+ str(random.randint(1,400000))+'\n')

In [29]:
with open('files/test_cold_user.csv','w') as f:
    for i in cold_start_user_triplet:
        f.write(   str(i[0]+1)+','+str(i[1]+1)+','+str(R[i[0],i[1]])+ ','+ str(random.randint(1,400000))+'\n')

In [30]:
with open('files/test_warm.csv','w') as f:
    for i in warm_test:
        f.write(   str(i[0]+1)+','+str(i[1]+1)+','+str(R[i[0],i[1]])+ ','+ str(random.randint(1,400000))+'\n')

In [31]:
with open('files/train.csv','w') as f:
    for i in train:
        f.write(   str(i[0]+1)+','+str(i[1]+1)+','+str(R[i[0],i[1]])+ ','+ str(random.randint(1,400000))+'\n')

In [32]:
with open('files/test_cold_item_item_ids.csv','w') as f:
    for i in cold_start_item_triplet:
        f.write(str(i[1]+1)+'\n')
        
with open('files/test_cold_user_item_ids.csv','w') as f:
    for i in cold_start_user_triplet:
        f.write(str(i[1]+1)+'\n')
        
with open('files/test_warm_item_ids.csv','w') as f:
    for i in warm_test:
        f.write(str(i[1]+1)+'\n')

In [35]:
# save_obj(warm_test,'warm_test')
# save_obj(cold_start_item_triplet,'cold_item')
# save_obj(cold_start_user_triplet,'cold_user')
# save_obj(train,'train')
# save_obj(R_copy,'R_train')
# save_obj(R,'R_full')

In [13]:
lines = open('files/train.csv').readlines()
x = []
y = []
rat = []
for line in lines:
    line_parts = line.split(',')
    x.append(int(line_parts[0])-1)
    y.append(int(line_parts[1])-1)
    rat.append(int(line_parts[2]))

R_train = csr_matrix((rat,(x, y)), dtype=np.int, shape=(1326101, 174567))

print len(R_train.nonzero()[0]),len(x)

4801323 4801323


In [53]:
# 1,15733,4
# R[0,15732] should be 4
print R_train[0,15732]

4


In [54]:
print np.matmul(user_mat[0,:] ,item_mat.T[:,15372])

0.015262493743269774


In [16]:
print(item_mat.shape,user_mat.shape,R_train.shape)

((174567, 50), (1326101, 50), (1326101, 174567))


In [22]:
# R = user_mat * item_mat
R_pred = np.matmul(user_mat[:,:], item_mat.T[:,:])

MemoryError: 

In [23]:
train_list = R_train.nonzero()

In [36]:
e = 0
count = 0
for i,j in zip(train_list[0],train_list[1]):
    count+=1
    if count%500000==1:
        print ('1/4')
    e+=(R_train[i,j] - np.matmul(user_mat[i,:] ,item_mat.T[:,j]))**2

1/4
1/4
1/4
1/4
1/4
1/4
1/4
1/4
1/4
1/4


In [33]:
item_mat.shape

(174567, 50)

In [30]:
R_train.shape

(1326101, 174567)

In [37]:
print np.sqrt(e/float(count))

3.784617985113


In [50]:
tmp = np.array([[1,2],[3,4]])

In [51]:
print tmp

[[1 2]
 [3 4]]


In [52]:
print tmp.T

[[1 3]
 [2 4]]


In [73]:
t1 = '-0.126172 0.00669824 0.0856417 -0.00666537 0.0482466 0.19337 0.0489063 -0.175737 -0.0421224 -0.0472006 -0.0071823 0.0105873 0.180058 -0.0835943 -0.0381727 0.0816491 0.0131445 0.00173505 -0.0175601 -0.0189759 0.0346226 -0.0350418 0.0854079 0.048259 0.0804236 -0.0150052 0.0235103 -0.0372168 -0.0190828 0.0860885 -0.034538 -0.0690639 -0.0384557 0.0798686 0.125233 0.0755056 -0.164857 0.0269163 -0.0200484 -0.0045255 0.022784 0.0794056 -0.12472 0.0664455 -0.160448 -0.0120457 -0.0943106 -0.0166067 -0.0430007 -0.0179245'
t1 = np.asarray(t1.split(),dtype = np.float64)
t2 = '-2.29444 -0.190278 -0.141641 -2.83498 0.216228 0.0151211 -0.334125 0.056945 0.295955 0.242633 0.261271 -0.133781 -0.275056 0.992787 -0.61381 0.398392 0.580795 0.0153873 0.302425 0.71524 -0.188693 0.389066 0.0564784 0.0259949 0.0304832 -0.522227 0.339366 -0.8946 -0.326001 -0.00365818 0.363473 -0.0509826 0.540101 0.529359 -0.402325 0.081239 -0.447601 0.0248618 0.561501 0.802014 -0.888769 -0.339893 -0.428042 0.023777 0.467606 -0.792586 -0.254688 -0.10963 0.475922 -1.27695'
t2 = np.asarray(t2.split(),dtype = np.float64)
print np.matmul(t1,t2)

0.20868959357697497


In [91]:
print user_mat[0,:].shape,item_mat.T[:,15372].shape


(50,) (50,)


In [89]:
print np.dot(user_mat[0,:],item_mat.T[:,15372])

0.015262493743269774


In [84]:
R_train[19,113332] # 20,113333,4,154155 line 63

4

In [86]:
print np.matmul(user_mat[19,:],item_mat.T[:,113332])

0.0795220291808938


In [88]:
item_mat.T[:,113332]

array([ 0.38905099,  0.0440753 , -0.125948  , -0.28595001, -0.17746501,
       -0.103179  ,  0.35663101,  0.71668297, -0.14459901, -0.81481397,
        1.07113004, -0.357281  , -0.0234064 ,  0.56392998,  0.0519976 ,
       -0.118972  , -0.0275773 ,  0.0289079 , -0.18492299, -0.60428602,
       -0.47387901,  0.89310497, -0.107237  ,  0.35916501,  0.0971235 ,
       -0.64525098,  1.01301003, -0.57667601, -0.84869498,  0.0603798 ,
       -0.36735001, -0.44551101, -0.24007   , -0.30570501, -0.0586973 ,
        0.45509499,  0.72814202, -0.0356238 ,  0.25022   ,  0.16923299,
       -0.55735898,  0.75028402, -0.0839065 ,  0.82292002,  0.383167  ,
       -0.52037197,  0.40407801,  0.71022499, -0.327209  , -0.333763  ])

In [87]:
user_mat[19,:]

array([ 0.0280626 , -0.46173999, -0.16196799,  0.0707588 , -0.244177  ,
        0.21941   ,  0.108033  ,  0.0129072 , -0.195823  ,  0.208158  ,
        0.147588  , -0.118327  , -0.237838  , -0.16970401, -0.14234801,
       -0.166539  , -0.22200599,  0.0327778 , -0.142409  , -0.00920581,
       -0.205639  ,  0.23772299, -0.111075  ,  0.31029901,  0.0234277 ,
        0.204748  ,  0.198129  ,  0.172621  ,  0.165396  , -0.46042499,
       -0.18631101,  0.364934  ,  0.31565401,  0.34031001,  0.125149  ,
       -0.35950699,  0.100071  , -0.164794  , -0.197124  ,  0.0241293 ,
       -0.0435426 ,  0.143663  , -0.0582582 ,  0.209361  , -0.18351001,
        0.15114699, -0.0399747 ,  0.0766665 ,  0.21115901, -0.136861  ])

In [79]:
np.linalg.norm(t1-user_mat[0,:])

1.32221975491854e-08

In [80]:
t1

array([-0.126172  ,  0.00669824,  0.0856417 , -0.00666537,  0.0482466 ,
        0.19337   ,  0.0489063 , -0.175737  , -0.0421224 , -0.0472006 ,
       -0.0071823 ,  0.0105873 ,  0.180058  , -0.0835943 , -0.0381727 ,
        0.0816491 ,  0.0131445 ,  0.00173505, -0.0175601 , -0.0189759 ,
        0.0346226 , -0.0350418 ,  0.0854079 ,  0.048259  ,  0.0804236 ,
       -0.0150052 ,  0.0235103 , -0.0372168 , -0.0190828 ,  0.0860885 ,
       -0.034538  , -0.0690639 , -0.0384557 ,  0.0798686 ,  0.125233  ,
        0.0755056 , -0.164857  ,  0.0269163 , -0.0200484 , -0.0045255 ,
        0.022784  ,  0.0794056 , -0.12472   ,  0.0664455 , -0.160448  ,
       -0.0120457 , -0.0943106 , -0.0166067 , -0.0430007 , -0.0179245 ])

In [81]:
user_mat[0,:]

array([-0.12617201,  0.00669824,  0.0856417 , -0.00666537,  0.0482466 ,
        0.19337   ,  0.0489063 , -0.17573699, -0.0421224 , -0.0472006 ,
       -0.0071823 ,  0.0105873 ,  0.180058  , -0.0835943 , -0.0381727 ,
        0.0816491 ,  0.0131445 ,  0.00173505, -0.0175601 , -0.0189759 ,
        0.0346226 , -0.0350418 ,  0.0854079 ,  0.048259  ,  0.0804236 ,
       -0.0150052 ,  0.0235103 , -0.0372168 , -0.0190828 ,  0.0860885 ,
       -0.034538  , -0.0690639 , -0.0384557 ,  0.0798686 ,  0.12523299,
        0.0755056 , -0.164857  ,  0.0269163 , -0.0200484 , -0.0045255 ,
        0.022784  ,  0.0794056 , -0.12472   ,  0.0664455 , -0.160448  ,
       -0.0120457 , -0.0943106 , -0.0166067 , -0.0430007 , -0.0179245 ])

In [60]:
item_mat[15732,:]

array([-2.29444003, -0.19027799, -0.14164101, -2.83498001,  0.21622799,
        0.0151211 , -0.33412501,  0.056945  ,  0.295955  ,  0.242633  ,
        0.261271  , -0.133781  , -0.275056  ,  0.992787  , -0.61381   ,
        0.39839199,  0.58079499,  0.0153873 ,  0.302425  ,  0.71524   ,
       -0.188693  ,  0.38906601,  0.0564784 ,  0.0259949 ,  0.0304832 ,
       -0.52222699,  0.33936599, -0.89459997, -0.32600099, -0.00365818,
        0.363473  , -0.0509826 ,  0.54010099,  0.52935898, -0.402325  ,
        0.081239  , -0.44760099,  0.0248618 ,  0.56150103,  0.80201399,
       -0.88876897, -0.33989301, -0.42804199,  0.023777  ,  0.46760601,
       -0.79258603, -0.25468799, -0.10963   ,  0.47592199, -1.27695   ])

In [77]:
t2

array([-2.29444   , -0.190278  , -0.141641  , -2.83498   ,  0.216228  ,
        0.0151211 , -0.334125  ,  0.056945  ,  0.295955  ,  0.242633  ,
        0.261271  , -0.133781  , -0.275056  ,  0.992787  , -0.61381   ,
        0.398392  ,  0.580795  ,  0.0153873 ,  0.302425  ,  0.71524   ,
       -0.188693  ,  0.389066  ,  0.0564784 ,  0.0259949 ,  0.0304832 ,
       -0.522227  ,  0.339366  , -0.8946    , -0.326001  , -0.00365818,
        0.363473  , -0.0509826 ,  0.540101  ,  0.529359  , -0.402325  ,
        0.081239  , -0.447601  ,  0.0248618 ,  0.561501  ,  0.802014  ,
       -0.888769  , -0.339893  , -0.428042  ,  0.023777  ,  0.467606  ,
       -0.792586  , -0.254688  , -0.10963   ,  0.475922  , -1.27695   ])

In [78]:
np.linalg.norm(item_mat[15732,:]-t2)

7.7704211809324e-08